<a href="https://colab.research.google.com/github/shaqas86/OpenAI_SDK/blob/main/HandsOff_Esclation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install openai-agents SDK


In [ ]:
!pip install -Uq openai-agents

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from google.colab import userdata

In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [ ]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

#HANDSOFF FUNCTION

In [ ]:
from pydantic import BaseModel

from agents import Agent, handoff, RunContextWrapper

class EscalationData(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: EscalationData):
    print(f"Escalation agent called with reason: {input_data.reason}")

agent = Agent(name="Escalation agent",model=model)

handoff_obj = handoff(
    agent=agent,
    on_handoff=on_handoff,
    input_type=EscalationData,
)

#Enable Logging

In [ ]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [ ]:
async def run_escalation():
    # Await the result of the asynchronous Runner.run method
    result = await Runner.run(agent, input="escalate")
    print(result)
    print(result.final_output)

# Run the async function
asyncio.run(run_escalation())

In [ ]:
from agents import Agent, handoff
from agents.extensions import handoff_filters

agent = Agent(name="FAQ agent",model=model)

handoff_obj = handoff(
    agent=agent,
    input_filter=handoff_filters.remove_all_tools,
)

In [ ]:
async def run_escalation():
    # Await the result of the asynchronous Runner.run method
    result = await Runner.run(agent, input="escalate")
    print(result)
    print(result.final_output)

# Run the async function
asyncio.run(run_escalation())